<a href="https://colab.research.google.com/github/ShraddhaSharma24/Natural-Language-Processing/blob/main/Knowledge_Graph_Backed_QA_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install faiss-cpu
!pip install rdflib
!pip install pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 6.0 MB/s eta 0:00:00


 Knowledge Graph

In [ ]:
import rdflib

# Load DBpedia knowledge graph (RDF format)
graph = rdflib.Graph()
graph.parse("http://dbpedia.org/resource/DBpedia")  # This loads DBpedia RDF data
print(f"Graph contains {len(graph)} triples.")


Graph contains 264 triples.


Text Preprocessing & Entity Linking

In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Load the pre-trained BERT model and tokenizer
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

def preprocess_text(question, context):
    # Tokenizing the question and context for BERT
    inputs = tokenizer(question, context, return_tensors="pt")
    return inputs

# Example question and context
question = "Who is the CEO of Tesla?"
context = "Elon Musk is the CEO of Tesla. He is known for his ventures in the tech industry."

inputs = preprocess_text(question, context)
print(inputs)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'input_ids': tensor([[  101,  2040,  2003,  1996,  5766,  1997, 26060,  1029,   102,  3449,
          2239, 14163,  6711,  2003,  1996,  5766,  1997, 26060,  1012,  2002,
          2003,  2124,  2005,  2010, 13252,  1999,  1996,  6627,  3068,  1012,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}


Answer Extraction Using BERT

In [ ]:
def get_answer(inputs):
    # Run the model and extract the answer
    with torch.no_grad():
        outputs = model(**inputs)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

    # Get the start and end token indices of the answer
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    # Convert token indices to actual text
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index+1]))
    return answer

answer = get_answer(inputs)
print(f"Answer: {answer}")


Answer: elon musk


Knowledge Graph Augmentation

In [ ]:
def enrich_answer_with_graph(answer, graph, question):
    # Simple example: Search the knowledge graph for entities related to the question
    # Example query: "Tesla" -> "CEO of Tesla"
    query = """
SELECT ?ceo
WHERE {
    <http://dbpedia.org/resource/Tesla> <http://www.w3.org/2000/01/rdf-schema#label> ?ceo.
}
"""

    results = graph.query(query)

    # Check if there is any information in the knowledge graph
    graph_answer = ""
    for row in results:
        graph_answer = row.ceo  # Extract the CEO information
    return graph_answer

graph_answer = enrich_answer_with_graph(answer, graph, question)
print(f"Graph-based Answer: {graph_answer}")


Graph-based Answer: 


In [ ]:
pip install SPARQLWrapper

In [ ]:
import spacy
from transformers import BertTokenizer, BertForQuestionAnswering
from SPARQLWrapper import SPARQLWrapper, JSON

# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased')

# SPARQL endpoint to query DBpedia for Tesla's CEO
sparql = SPARQLWrapper("http://dbpedia.org/sparql")

def get_knowledge_graph_answer(query):
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    answer = results["results"]["bindings"]
    return answer[0]["ceo"]["value"] if answer else "No information found"

# Define a function for QA using BERT
def get_bert_answer(question, context):
    inputs = tokenizer.encode(question, context)
    tokens = tokenizer.convert_ids_to_tokens(inputs)
    start_scores, end_scores = model(torch.tensor([inputs]))

    # Find the position of the answer
    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)
    answer_tokens = tokens[start_index:end_index + 1]
    answer = tokenizer.convert_tokens_to_string(answer_tokens)
    return answer

# Example question
question = "Who is the CEO of Tesla?"
context = "Tesla is an electric vehicle and clean energy company."

# Step 1: Get answer from BERT
bert_answer = get_bert_answer(question, context)

# Step 2: Query the knowledge graph for factual information
query = """
SELECT ?ceo WHERE {
    <http://dbpedia.org/resource/Tesla> <http://dbpedia.org/ontology/CEO> ?ceo.
}
"""
kg_answer = get_knowledge_graph_answer(query)

# Step 3: Combine BERT answer with KG answer
final_answer = f"Answer from BERT: {bert_answer}\nEnriched with Knowledge Graph: {kg_answer}"
print(final_answer)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: argmax(): argument 'input' (position 1) must be Tensor, not str

In [ ]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace

# Define a namespace
MH = Namespace("http://example.org/mentalhealth/")

# Create a graph
kg = Graph()

# Define entities
depression = URIRef(MH["Depression"])
anxiety = URIRef(MH["Anxiety"])
therapy = URIRef(MH["Therapy"])
cbt = URIRef(MH["CBT"])
medication = URIRef(MH["Medication"])

# Define relationships
kg.add((depression, RDF.type, MH["MentalHealthDisorder"]))
kg.add((depression, MH["treatments"], cbt))
kg.add((depression, MH["treatments"], medication))

kg.add((anxiety, RDF.type, MH["MentalHealthDisorder"]))
kg.add((anxiety, MH["treatments"], cbt))

kg.add((cbt, RDF.type, MH["Therapy"]))
kg.add((cbt, MH["isEffectiveFor"], depression))
kg.add((cbt, MH["isEffectiveFor"], anxiety))


print(kg.serialize(format="turtle"))



@prefix ns1: <http://example.org/mentalhealth/> .

ns1:Anxiety a ns1:MentalHealthDisorder ;
    ns1:treatments ns1:CBT .

ns1:Depression a ns1:MentalHealthDisorder ;
    ns1:treatments ns1:CBT,
        ns1:Medication .

ns1:CBT a ns1:Therapy ;
    ns1:isEffectiveFor ns1:Anxiety,
        ns1:Depression .




Query the KG (e.g., “What treats depression?”)

In [ ]:
def query_kg_disorder_treatments(disorder_name):
    query = f"""
    PREFIX mh: <http://example.org/mentalhealth/>

    SELECT ?treatment
    WHERE {{
        mh:{disorder_name} mh:treatments ?treatment .
    }}
    """
    results = kg.query(query)
    return [str(row.treatment).split("/")[-1] for row in results]


Base BERT QA Answer

In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

def get_bert_answer(question, context):
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt")
    input_ids = inputs["input_ids"]
    token_type_ids = inputs["token_type_ids"]

    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=token_type_ids)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores) + 1

    answer_tokens = input_ids[0][start_index:end_index]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
    return answer


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Combine Both

In [ ]:
def mental_health_qa(question, context, disorder="Depression"):
    bert_answer = get_bert_answer(question, context)
    kg_answer = query_kg_disorder_treatments(disorder)

    print("🧠 BERT Answer:", bert_answer)
    print("🔗 Knowledge Graph Enrichment:", ", ".join(kg_answer))

    # Optionally fuse both answers
    return f"{bert_answer}. Additional evidence-based treatments: {', '.join(kg_answer)}."


In [ ]:
context = "Depression is a mental disorder that affects mood and behavior. Therapy and medication are often recommended."
question = "How can depression be treated?"

final_answer = mental_health_qa(question, context, disorder="Depression")
print("✅ Final Answer:", final_answer)


🧠 BERT Answer: therapy and medication
🔗 Knowledge Graph Enrichment: CBT, Medication
✅ Final Answer: therapy and medication. Additional evidence-based treatments: CBT, Medication.


In [ ]:
pip install rdflib


In [ ]:
from rdflib import Graph, Namespace, URIRef, Literal, RDF

# Define a namespace
CYBER = Namespace("http://example.org/cyber#")

# Create a graph
g = Graph()
g.bind("cyber", CYBER)

# Define URIs
phishing = URIRef(CYBER.Phishing)
malware = URIRef(CYBER.Malware)
two_factor_auth = URIRef(CYBER.TwoFactorAuthentication)
user_awareness = URIRef(CYBER.UserAwarenessTraining)
antivirus = URIRef(CYBER.AntiVirus)

# Add types and relationships
g.add((phishing, RDF.type, CYBER.AttackVector))
g.add((malware, RDF.type, CYBER.AttackVector))

g.add((phishing, CYBER.preventedBy, two_factor_auth))
g.add((phishing, CYBER.preventedBy, user_awareness))
g.add((malware, CYBER.preventedBy, antivirus))

g.add((two_factor_auth, CYBER.description, Literal("Use two-factor authentication for login.")))
g.add((user_awareness, CYBER.description, Literal("Conduct phishing awareness training.")))
g.add((antivirus, CYBER.description, Literal("Install and update antivirus software.")))

# Show the graph in Turtle format
print(g.serialize(format="turtle"))



@prefix cyber: <http://example.org/cyber#> .

cyber:Malware a cyber:AttackVector ;
    cyber:preventedBy cyber:AntiVirus .

cyber:Phishing a cyber:AttackVector ;
    cyber:preventedBy cyber:TwoFactorAuthentication,
        cyber:UserAwarenessTraining .

cyber:AntiVirus cyber:description "Install and update antivirus software." .

cyber:TwoFactorAuthentication cyber:description "Use two-factor authentication for login." .

cyber:UserAwarenessTraining cyber:description "Conduct phishing awareness training." .




In [ ]:
from rdflib.plugins.sparql import prepareQuery

# Define SPARQL query to find prevention techniques for phishing
query = prepareQuery("""
    PREFIX cyber: <http://example.org/cyber#>
    SELECT ?prevention ?desc WHERE {
        cyber:Phishing cyber:preventedBy ?prevention .
        ?prevention cyber:description ?desc .
    }
""")

# Run query
results = g.query(query)

# Print results
print("✅ Prevention Techniques for Phishing:\n")
for row in results:
    print(f"- {row.prevention.split('#')[-1]}: {row.desc}")


✅ Prevention Techniques for Phishing:

- TwoFactorAuthentication: Use two-factor authentication for login.
- UserAwarenessTraining: Conduct phishing awareness training.


In [ ]:
!pip install transformers
!pip install rdflib
!pip install torch


In [ ]:
from rdflib import Graph, Namespace, URIRef, Literal, RDF

# Namespaces
CYBER = Namespace("http://example.org/cyber#")

# Create graph
g = Graph()

# Add example KG triples
phishing = URIRef(CYBER.Phishing)
tfa = URIRef(CYBER.TwoFactorAuthentication)
training = URIRef(CYBER.UserAwarenessTraining)

g.add((phishing, CYBER.preventedBy, tfa))
g.add((phishing, CYBER.preventedBy, training))
g.add((tfa, CYBER.description, Literal("Use two-factor authentication for login.")))
g.add((training, CYBER.description, Literal("Conduct phishing awareness training.")))

# Save the graph to file
g.serialize(destination="cybersecurity_kg.ttl", format="turtle")
print("✅ Knowledge Graph saved as 'cybersecurity_kg.ttl'")


✅ Knowledge Graph saved as 'cybersecurity_kg.ttl'


In [ ]:
from transformers import pipeline
from rdflib import Graph
from rdflib.plugins.sparql import prepareQuery

# Load the summarization/QA pipeline
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

# Load the RDF Knowledge Graph
g = Graph()
g.parse("cybersecurity_kg.ttl", format="turtle")

# Example cybersecurity context for QA model
context = """
Phishing attacks trick users into revealing personal info. Two-factor authentication adds an extra layer of security.
Regular employee training can reduce risk of cyber threats.
"""

# === Step 1: Get answer from BERT ===
def get_bert_answer(question, context):
    result = qa_pipeline(question=question, context=context)
    return result["answer"]

# === Step 2: Query KG for enrichment ===
def get_kg_enrichment(question):
    if "phishing" in question.lower():
        query = prepareQuery("""
            PREFIX cyber: <http://example.org/cyber#>
            SELECT ?prevention ?desc WHERE {
                cyber:Phishing cyber:preventedBy ?prevention .
                ?prevention cyber:description ?desc .
            }
        """)
        results = g.query(query)
        enrichments = [f"{row.prevention.split('#')[-1]}: {row.desc}" for row in results]
        return enrichments
    else:
        return ["(No relevant KG knowledge found.)"]

# === Step 3: Ask Question ===
question = "How to prevent phishing?"
bert_answer = get_bert_answer(question, context)
kg_info = get_kg_enrichment(question)

# === Final Enriched Answer ===
print(f"\n📌 Question: {question}")
print(f"🤖 BERT Answer: {bert_answer}")
print("🧠 Knowledge Graph Adds:")
for fact in kg_info:
    print(f"  - {fact}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you ar


📌 Question: How to prevent phishing?
🤖 BERT Answer: Regular employee training
🧠 Knowledge Graph Adds:
  - TwoFactorAuthentication: Use two-factor authentication for login.
  - UserAwarenessTraining: Conduct phishing awareness training.
